In [5]:
import json
import time
import datetime

from episode_mining.winepi import WINEPI, WinEpiRules

In [6]:
sequence = sorted([
    (31, 'E'), (31, 'D'), (32, 'A'), (32, 'B'),(33, 'E'), (33, 'D'), (34, 'A'), (34, 'B'),
], key=lambda x:x[0])

sequence = sorted([
    (31, 'E'), (31, 'D'), (33, 'E'), (33, 'D'), (33, 'E'), (58, 'E'), (58, 'D')
], key=lambda x:x[0])

sequence = sorted([
    (31, 'E'),(31, 'X'),(31, 'Y'),(31, 'Z'), (31, 'D'), (32, 'A'), (32, 'B'),(35, 'E'), (35, 'D'), (36, 'A'), (36, 'B'),
], key=lambda x:x[0])

In [7]:
alg = WINEPI(sequence, episode_type='serial')
freqItems, suppData = alg.WinEpi(width=3, step=1, minFrequent=0.1)

Generating windows between [0, 7 )
Processing Window 0
Processing Window 1
Processing Window 2
Processing Window 3
Processing Window 4
Processing Window 5
Processing Window 6
Finished Window creation, beginning c1
Finished Window 0 total: 8
Finished Window 1 total: 8
Finished Window 2 total: 8
Finished Window 3 total: 8
Finished Window 4 total: 8
Finished Window 5 total: 8
Finished Window 6 total: 8
Finished Window 7 total: 8
Finished C1 creation, beginning window scan
Starting Serial Window Scan
Starting Window 0 Total: 8
Starting Window 1 Total: 8
Starting Window 2 Total: 8
Starting Window 3 Total: 8
Starting Window 4 Total: 8
Starting Window 5 Total: 8
Starting Window 6 Total: 8
Starting Window 7 Total: 8
Finished First Window scan, beginning k=2
2
Finished apriori Gen 2 beginning window scan
Starting Serial Window Scan
Starting Window 0 Total: 8


KeyboardInterrupt: 

In [5]:
winepiRules = WinEpiRules(freqItems, suppData, width=3, minConfidence=0.7)
ruleList = winepiRules.generateRules()
winepiRules.printRules(ruleList)

WINEPIRule: ['A'] ==> ['A', 'B'] [3] [0.75, 1.0]
WINEPIRule: ['B'] ==> ['A', 'B'] [3] [0.75, 1.0]
WINEPIRule: ['Z'] ==> ['Z', 'D'] [3] [0.375, 1.0]
WINEPIRule: ['Y'] ==> ['Y', 'Z'] [3] [0.375, 1.0]
WINEPIRule: ['Z'] ==> ['Y', 'Z'] [3] [0.375, 1.0]
WINEPIRule: ['Y'] ==> ['Y', 'D'] [3] [0.375, 1.0]
WINEPIRule: ['X'] ==> ['X', 'Z'] [3] [0.375, 1.0]
WINEPIRule: ['Z'] ==> ['X', 'Z'] [3] [0.375, 1.0]
WINEPIRule: ['X'] ==> ['X', 'Y'] [3] [0.375, 1.0]
WINEPIRule: ['Y'] ==> ['X', 'Y'] [3] [0.375, 1.0]
WINEPIRule: ['X'] ==> ['X', 'D'] [3] [0.375, 1.0]
WINEPIRule: ['Z'] ==> ['E', 'Z'] [3] [0.375, 1.0]
WINEPIRule: ['Y'] ==> ['E', 'Y'] [3] [0.375, 1.0]
WINEPIRule: ['X'] ==> ['E', 'X'] [3] [0.375, 1.0]
WINEPIRule: ['E'] ==> ['E', 'D'] [3] [0.75, 1.0]
WINEPIRule: ['D'] ==> ['E', 'D'] [3] [0.75, 1.0]
